# Download and store STOOQ data

This notebook contains information on downloading the STOOQ stock and ETF price data that we use in [Chapter 09](../09_time_series_models) for a pairs trading strategy based on cointegration and [Chapter 11](../11_decision_trees_random_forests) for a long-short strategy using Random Forest return predictions.

## Imports & Settings

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from pathlib import Path
import requests
from io import BytesIO
from zipfile import ZipFile, BadZipFile

import numpy as np
import pandas as pd
import pandas_datareader.data as web
from sklearn.datasets import fetch_openml

pd.set_option('display.expand_frame_repr', False)

## Set Data Store path

Modify the path to the `DATA_STORE` if you would like to store the data elsewhere and change the notebooks accordingly

In [21]:
DATA_STORE = Path('assets.h5')

## Stooq Historical Market Data

> Note that the below downloading details may change at any time as Stooq updates their website; if you encounter errors, please inspect their website and raise a GitHub issue to let us know so we can update the information.

> Update 12/2020: please note that STOOQ will disable automatic downloads and require CAPTCHA starting Dec 10, 2020 so that the code that downloads and unpacks the zip files will no longer work; please navigate to their website [here](https://stooq.com/db/h/) for manual download.

### Download price data

1. Download **price data** for the selected combination of asset class, market and frequency from [the Stooq website](https://stooq.com/db/h/)
2. Store the result under `stooq` using the preferred folder structure outlined on the website. It has the structure: `/data/freq/market/asset_class`, such as `/data/daily/us/nasdaq etfs`.

In [22]:
stooq_path = Path('stooq') 
if not stooq_path.exists():
    stooq_path.mkdir()

Use the symbol for the market you want to download price data for. In this book we'll be useing `us` and `jp`. 

In [23]:
STOOQ_URL = 'https://static.stooq.com/db/h/'


In [8]:
def download_price_data(market='us',
                        freq = 'd'):
    data_name = stooq_path.joinpath('.downloaded',f'{freq}_{market}_txt.zip')
    if not data_name.exists():
        raise FileNotFoundError(f'You must download the data first. {data_name} not found.')
    with ZipFile(data_name) as zip_file:
        for i, file in enumerate(zip_file.namelist()):
            if not file.endswith('.txt'):
                continue
            local_file = stooq_path / file
            local_file.parent.mkdir(parents=True, exist_ok=True)
            try:
                with local_file.open('wb') as output:
                    for line in zip_file.open(file).readlines():
                        output.write(line)

            except FileNotFoundError as fnfe:
                print(fnfe)
                continue

In [ ]:
for market in ['us', 'jp']:
    download_price_data(market=market)

### Add symbols

Add the corresponding **symbols**, i.e., tickers and names by following the directory tree on the same site. You can also adapt the following code snippet using the appropriate asset code that you find by inspecting the url; this example works for NASDAQ ETFs that have code `g=69`:
```python
df = pd.read_csv('https://stooq.com/db/l/?g=69', sep='        ').apply(lambda x: x.str.strip())
df.columns = ['ticker', 'name']
df.drop_duplicates('ticker').to_csv('stooq/data/tickers/us/nasdaq etfs.csv', index=False)
```

In [11]:
metadata_dict = {
    ('jp', 'tse etfs'): 34,
    ('jp', 'tse stocks'): 32,
    ('us', 'nasdaq etfs'): 69,
    ('us', 'nasdaq stocks'): 27,
    ('us', 'nyse etfs'): 70,
    ('us', 'nyse stocks'): 28,
    ('us', 'nysemkt stocks'): 26
}

metadata_dict = {
    ('us', 'nasdaq etfs','d'): 70,
}

In [12]:
meta_path = Path('stooq') / 'data/meta'
for (market, asset_class,freq), code in metadata_dict.items():
    path = meta_path / f'{freq}_{market}_ms/data/daily/{market}/{asset_class}/EMASTER'
    df = pd.read_csv(path, sep=r'\s+').apply(lambda x: x.str.strip())
    df.columns = ['ticker', 'name']
    df = df.drop_duplicates('ticker').dropna()
    print(market, asset_class, f'# tickers: {df.shape[0]:,.0f}')
    path = stooq_path / 'tickers' / market
    if not path.exists():
        path.mkdir(parents=True)
    df.to_csv(path / f'{asset_class}.csv', index=False)    

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

### Store price data in HDF5 format

To speed up loading, we store the price data in HDF format. The function `get_stooq_prices_and_symbols` loads data assuming the directory structure described above and takes the following arguments:
- frequency (see Stooq website for options as these may change; default is `daily`
- market (default: `us`), and 
- asset class (default: `nasdaq etfs`.

It removes files that do not have data or do not appear in the corresponding list of symbols.

In [31]:
def get_stooq_prices_and_tickers(frequency='daily',
                                 market='us',
                                 asset_class='nasdaq etfs'):
    prices = []


    if frequency in ['5 min', 'hourly']:
        parse_dates = [['date', 'time']]
        date_label = 'date_time'
    else:
        parse_dates = ['date']
        date_label = 'date'
    names = ['ticker', 'freq', 'date', 'time', 
             'open', 'high', 'low', 'close','volume', 'openint']
    
    usecols = ['ticker', 'open', 'high', 'low', 'close', 'volume'] + parse_dates
    path = stooq_path / 'data/data' / frequency / market / asset_class
    print(path.as_posix())
    files = path.glob('**/*.txt')
    tickers = {}
    for i, file in enumerate(files, 1):
        if i % 500 == 0:
            print(i)
        try:
            df = (pd.read_csv(
                file,
                names=names,
                usecols=usecols,
                header=0,
                parse_dates=parse_dates))
            prices.append(df)
            tickers[df.ticker.iloc[0]] = (file.stem.split('.')[0].lower())
        except (pd.errors.EmptyDataError, IndexError):
            print('\tdata missing', file.stem)
            #file.unlink()

    prices = (pd.concat(prices, ignore_index=True)
              .rename(columns=str.lower)
              .set_index(['ticker', date_label])
              .apply(lambda x: pd.to_numeric(x, errors='coerce')))
    return prices,pd.Series(tickers,name = 'ticker')

We'll be using US equities and ETFs in [Chapter 9](../09_time_series_models) and and Japanese equities in [Chapter 11](../11_decision_trees_random_forests). The following code collects the price data for the period 2000-2019 and stores it with the corresponding symbols in the global `assets.h5` store:

In [32]:
# load some Japanese and all US assets for 2000-2019
#markets = {'jp': ['tse stocks'],
#           'us': ['nasdaq etfs', 'nasdaq stocks', 'nyse etfs', 'nyse stocks', 'nysemkt stocks']
#          }
markets = {'us': ['nasdaq etfs', 'nasdaq stocks', 'nyse etfs', 'nyse stocks', 'nysemkt stocks']}

frequency = 'daily'

idx = pd.IndexSlice
for market, asset_classes in markets.items():
    for asset_class in asset_classes:
        print(f'\n{asset_class}')
        prices,tickers = get_stooq_prices_and_tickers(frequency=frequency, 
                                                       market=market, 
                                                       asset_class=asset_class)
        
        prices = prices.sort_index().loc[idx[:, '2000': '2019'], :]
        names = prices.index.names
        prices = (prices
                  .reset_index()
                  .drop_duplicates()
                  .set_index(names)
                  .sort_index())
        
        print('\nNo. of observations per asset')
        print(prices.groupby('ticker').size().describe())
        key = f'stooq/{market}/{asset_class.replace(" ", "/")}/'
        
        print(prices.info())
        
        prices.to_hdf(DATA_STORE, key + 'prices', format='t')
        
        tickers.to_hdf(DATA_STORE, key + 'tickers', format='t')


nasdaq etfs
stooq/data/data/daily/us/nasdaq etfs

No. of observations per asset
count     301.000000
mean     1732.063123
std      1135.174568
min         2.000000
25%       804.000000
50%      1677.000000
75%      2542.000000
max      5031.000000
dtype: float64
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 521351 entries, ('AADR.US', Timestamp('2010-07-21 00:00:00')) to ('YLDE.US', Timestamp('2019-12-27 00:00:00'))
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   open    521351 non-null  float64
 1   high    521351 non-null  float64
 2   low     521351 non-null  float64
 3   close   521351 non-null  float64
 4   volume  521351 non-null  float64
dtypes: float64(5)
memory usage: 22.1+ MB
None


C:\Users\mlanahan\AppData\Local\Temp\ipykernel_14628\2539056611.py:31: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  prices.to_hdf(DATA_STORE, key + 'prices', format='t')
C:\Users\mlanahan\AppData\Local\Temp\ipykernel_14628\2539056611.py:33: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  tickers.to_hdf(DATA_STORE, key + 'tickers', format='t')



nasdaq stocks
stooq/data/data/daily/us/nasdaq stocks
500
	data missing biya.us
1000
	data missing cubwu.us
	data missing ehldv.us
	data missing eseav.us
1500
	data missing fvnnu.us
2000
	data missing lifw.us
2500
3000
	data missing pirs.us
	data missing pqap.us
3500
4000
	data missing wtf.us
4500


C:\Users\mlanahan\AppData\Local\Temp\ipykernel_14628\3142032891.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  prices = (pd.concat(prices, ignore_index=True)



No. of observations per asset
count    2063.000000
mean     2273.589918
std      1511.723111
min         2.000000
25%       740.000000
50%      2355.000000
75%      3737.000000
max      5031.000000
dtype: float64
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4690416 entries, ('AACG.US', Timestamp('2008-01-28 00:00:00')) to ('ZYXI.US', Timestamp('2019-12-31 00:00:00'))
Data columns (total 5 columns):
 #   Column  Dtype  
---  ------  -----  
 0   open    float64
 1   high    float64
 2   low     float64
 3   close   float64
 4   volume  float64
dtypes: float64(5)
memory usage: 197.1+ MB
None


...

C:\Users\mlanahan\AppData\Local\Temp\ipykernel_14628\3142032891.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  prices = (pd.concat(prices, ignore_index=True)



No. of observations per asset
count    1088.000000
mean     2720.434743
std      1536.633498
min         2.000000
25%      1316.000000
50%      3555.500000
75%      3737.000000
max      5031.000000
dtype: float64
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2959833 entries, ('A.US', Timestamp('2000-01-03 00:00:00')) to ('KEX.US', Timestamp('2019-12-31 00:00:00'))
Data columns (total 5 columns):
 #   Column  Dtype  
---  ------  -----  
 0   open    float64
 1   high    float64
 2   low     float64
 3   close   float64
 4   volume  float64
dtypes: float64(5)
memory usage: 124.4+ MB
None


C:\Users\mlanahan\AppData\Local\Temp\ipykernel_14628\2539056611.py:31: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  prices.to_hdf(DATA_STORE, key + 'prices', format='t')
C:\Users\mlanahan\AppData\Local\Temp\ipykernel_14628\2539056611.py:33: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  tickers.to_hdf(DATA_STORE, key + 'tickers', format='t')



nysemkt stocks
stooq/data/data/daily/us/nysemkt stocks

No. of observations per asset
count     195.000000
mean     2700.430769
std      1265.881604
min         4.000000
25%      1652.000000
50%      3423.000000
75%      3728.500000
max      5029.000000
dtype: float64
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 526584 entries, ('ACCS.US', Timestamp('2008-03-10 00:00:00')) to ('ZOMDF.US', Timestamp('2019-12-31 00:00:00'))
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   open    526584 non-null  float64
 1   high    526584 non-null  float64
 2   low     526584 non-null  float64
 3   close   526584 non-null  float64
 4   volume  526584 non-null  float64
dtypes: float64(5)
memory usage: 22.3+ MB
None


C:\Users\mlanahan\AppData\Local\Temp\ipykernel_14628\2539056611.py:31: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  prices.to_hdf(DATA_STORE, key + 'prices', format='t')
C:\Users\mlanahan\AppData\Local\Temp\ipykernel_14628\2539056611.py:33: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  tickers.to_hdf(DATA_STORE, key + 'tickers', format='t')
